In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

import pandas as pd
from time import sleep



In [90]:
class KakaoMapScraper:
    '''
    카카오맵 리뷰 데이터를 크롤링하는 클래스
    '''
    def __init__(self):
        # webdriver path, 카카오맵 url 설정
        self.driver = webdriver.Chrome(executable_path="./chromedriver")
        self.url = "https://map.kakao.com"
        self.driver.implicitly_wait(1)

        self.driver.get(self.url)
        self.driver.implicitly_wait(5)
    
    def _get_place_address_list(self, keywords):
        '''
        키워드 검색 결과의 장소들의 상세보기 주소를 스크래핑하여 반환
        param: keywords (type: list)
        return: place_address_list (type: dict)
        '''
        
        # 검색창 찾기
        search_area = self.driver.find_element(By.ID, "search.keyword.query") #.find_element_by_id("search.keyword.query")
        place_address_list = {}
        
        for keyword in keywords:
            # 키워드 검색
            search_area.clear()
            search_area.send_keys(keyword)
            self.driver.find_element(By.ID, "search.keyword.submit").send_keys(Keys.ENTER)
            self.driver.implicitly_wait(2)
            
            try: # 더보기 버튼이 있을시 클릭하여 진행
                self.driver.find_element(By.ID, "info.search.place.more").send_keys(Keys.ENTER)
                self.driver.implicitly_wait(2)
                
            except: # 1페이지만 존재할 때
                place_address_list[keyword]= []
                html = self.driver.page_source
                soup = BeautifulSoup(html, "html.parser")
                places = soup.select("ul.placelist > li.PlaceItem.clickArea > div.info_item > div.contact.clickArea")
                        
                # 현재 페이지 장소들의 상세보기 주소를 저장
                for place in places:
                    place_address_list[keyword].append(place.select_one("a")["href"])

                continue
                
            
            # 여러 페이지 존재할 경우, 1페이지부터 장소들의 상세보기 주소를 가져옴
            try:
                place_address_list[keyword]= []
                page_num = 1
                while True:
                    try:
                        # 페이지 이동
                        self.driver.find_element(By.ID, f"info.search.page.no{page_num}").send_keys(Keys.ENTER)#  find_element_by_id(f"info.search.page.no{page_num}").send_keys(Keys.ENTER)
                        sleep(1)

                        html = self.driver.page_source
                        soup = BeautifulSoup(html, "html.parser")
                        places = soup.select("ul.placelist > li.PlaceItem.clickArea > div.info_item > div.contact.clickArea")
                        
                        # 현재 페이지 장소들의 상세보기 주소를 저장
                        for place in places:
                            place_address_list[keyword].append(place.select_one("a")["href"])

                    except:
                        break
                    
                    page_num += 1
                    
                    # 5페이지가 넘어가면 다음 버튼을 클릭
                    if page_num == 6:
                        page_num = 1
                        try:
                            self.driver.find_element(By.ID,"info.search.page.next").send_keys(Keys.ENTER)
                            self.driver.implicitly_wait(2)
                        except:
                            break
            except:
                break

        return place_address_list
    
    def get_data(self, keywords):
        '''
        각 장소별 리뷰 데이터를 크롤링
        params: keywords (type: list)
        return: review_df (type: pd.DataFrame)
        '''
        
        # 키워드 장소들의 상세보기 주소를 가져옴
        place_address_list = self._get_place_address_list(keywords)
        
        # 리뷰 데이터를 저장할 데이터프레임 생성
        review_df = pd.DataFrame(columns=["name", "average_rating", "user", "user_rating", "comment", "url", "target"])
        
        # 각 장소별 리뷰 데이터 크롤링
        for place, addresses in place_address_list.items():
            for address in addresses:
                # 각 장소의 상세보기 주소로 이동
                self.driver.get(address)
                sleep(2)

                html = self.driver.page_source
                soup = BeautifulSoup(html, "html.parser")
                
                # 장소 정보 저장(장소 이름, 분류, 평균 별점)
                place_info = soup.select_one("div.inner_place")
                #name = place_info.select_one("h2.tit_location").text
                #category = place_info.select_one("div.location_evaluation > span.txt_location").text[4:]
                average_rating = float(place_info.select_one("a.link_evaluation > span.color_b").text[:-1])
                location = self.driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div[2]/div[1]/div/span[1]').text
                print(location)


                # 주소가 "서울 마포구"가 아닐 경우 크롤링을 진행하지 않음
                if location[:6] != "서울 마포구": continue
                
                # 1페이지의 후기부터 크롤링
                # page_num = 1
                # while True:
                #     # 후기 더보기 버튼 있을시 클릭
                #     buttons = self.driver.find_elements(By.CLASS_NAME, "btn_fold")#  .find_elements_by_class_name("btn_fold")
                #     for button in buttons:
                #         if button.is_displayed() and button.is_enabled():
                #             button.send_keys(Keys.ENTER)
                #     sleep(0.5)
                    
                #     # html 파싱
                #     html = self.driver.page_source
                #     soup = BeautifulSoup(html, "html.parser")
                #     reviews = soup.select("ul.list_evaluation > li")
                    
                #     # 각 후기별 정보 저장(작성자 이름, 작성자 별점, 작성자 후기)
                #     for review in reviews:
                #         user = review.select_one("a")["data-username"]
                #         rating_per = review.select_one("div.star_info > div > span > span")# > div.grade_star size_s > span.ico_star star_rate > span.ico_star inner_star")
                #         #print(rating_per)
                #         rating_per = rating_per["style"][6:]
                #         rating_per = rating_per[:-2]
                #         user_rating = float(rating_per)/20
                #         comment = review.select_one("div.comment_info > p.txt_comment > span").text
                        
                #         review_data = {"name": place, 
                #                        "average_rating": average_rating,
                #                        "user": user,
                #                        "user_rating": user_rating,
                #                        "comment": comment,
                #                        "url": address}
                #         review_df = review_df.append(review_data, ignore_index=True)
                    
                #     # 다음 후기 페이지가 있으면 이동
                #     try:
                #         page_num += 1
                #         self.driver.find_element_by_css_selector(f"a[data-page='{page_num}']").click()
                #         sleep(1)
                #     except: 
                #         break

                # 후기 크롤링
                while True:
                    # 후기 더보기 버튼 끝까지 클릭
                    button = self.driver.find_element(By.CLASS_NAME, 'link_more')
                    sleep(0.5)
                    btn_text = button.text
                    
                    if btn_text == "메뉴 더보기":
                        button = self.driver.find_elements(By.CLASS_NAME, 'link_more')
                        sleep(0.5)
                        btn_text = button[1].text
                        button = button[1]

                    if btn_text == "후기 접기":
                        break

                    button.click()
                
                # html 파싱
                html = self.driver.page_source
                soup = BeautifulSoup(html, "html.parser")
                reviews = soup.select("ul.list_evaluation > li")
                
                # 각 후기별 정보 저장(작성자 이름, 작성자 별점, 작성자 후기)
                for review in reviews:
                    user = review.select_one("a")["data-username"]
                    rating_per = review.select_one("div.star_info > div > span > span")# > div.grade_star size_s > span.ico_star star_rate > span.ico_star inner_star")
                    #print(rating_per)
                    rating_per = rating_per["style"][6:]
                    rating_per = rating_per[:-2]
                    user_rating = float(rating_per)/20
                    comment = review.select_one("div.comment_info > p.txt_comment > span").text
                    target = 1 if user_rating >3 else 0
                    review_data = {"name": place, 
                                    "average_rating": average_rating,
                                    "user": user,
                                    "user_rating": user_rating,
                                    "comment": comment,
                                    "url": address,
                                    "target": target}
                    review_df = review_df.append(review_data, ignore_index=True)

        return review_df

In [91]:
scraper = KakaoMapScraper()

/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/468085606.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(executable_path="./chromedriver")


In [92]:
keywords = ["후계동", "카미야", "카레시", "국시와 가래떡", "지로우라멘", "하카타분코", "가미우동", "요이동", "금복식당"]
df = scraper.get_data(keywords)

서울 마포구 독막로 76 (우)04074


/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/468085606.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/468085606.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/468085606.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/468085606.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pa

서울 마포구 와우산로21길 28-6 지하1층 (우)04040


/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/468085606.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/468085606.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/468085606.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/468085606.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pa

서울 마포구 독막로9길 31 (우)04048


...

서울 마포구 독막로14길 24 지하1층 (우)04074


/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/468085606.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/468085606.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/468085606.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/468085606.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pa

서울 동대문구 안암로20길 16 (우)02473
인천 서구 검암로30번길 10 (우)22704
서울 강북구 인수봉로 195 (우)01023
경기 의정부시 의정로46번길 50-1 (우)11624
경기 성남시 수정구 수정로 259-1 1충 (우)13259
경기 시흥시 금오로309번길 22 2층 (우)14932
대전 중구 대전천서로 287 1층 (우)35029
충남 논산시 와야길 109 (우)32995
경북 문경시 매봉2길 25-20 (우)36983
대구 서구 국채보상로46길 7 1층 (우)41824
경북 안동시 도산면 퇴계로 2622 (우)36604
강원 양양군 양양읍 남문9길 19-1 (우)25030
경북 김천시 아랫장터2길 16 (우)39576
광주 서구 상무대로868번길 3-10 (우)61989
대구 동구 화랑로27길 36 (우)41241
대구 수성구 청호로96길 38 (우)42079
경북 경주시 중앙로29번길 8 (우)38154
제주특별자치도 제주시 동문로 16 (우)63264
전남 광양시 광양읍 읍내중앙길 57 (우)57744
제주특별자치도 서귀포시 중앙로48번길 18 (우)63591
전남 보성군 노동면 보림길 2 (우)59444
전남 광양시 진월면 선소중앙길 28 (우)57712
부산 서구 감천로 261 삼성비치타운 102호 (우)49265
전남 여수시 충무4길 6 (우)59733
부산 부산진구 신천대로204번길 31 (우)47194
경기 파주시 소리천로 25 2층 (우)10908
서울 강서구 마곡동로4길 15 앰팰리체 2층 (우)07803
인천 서구 중봉대로586번길 9-11 청라매그놀리아오피스텔 109,110호 (우)22736
서울 마포구 삼개로3길 6-3 1층 (우)04172


/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/468085606.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/468085606.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/468085606.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/468085606.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pa

경기 김포시 김포한강4로 8 라베니체마치에비뉴4차 2층 215,216호 (우)10090
서울 구로구 공원로6나길 17-1 (우)08289
서울 강북구 도봉로10가길 11 (우)01219
경기 광명시 오리로856번길 26 2층 (우)14240
서울 영등포구 영등포로 190 (우)07301
서울 서초구 강남대로65길 9 1층 (우)06614
서울 광진구 아차산로33길 31 (우)05017
서울 중구 퇴계로85길 42 1층 (우)04576
서울 송파구 양재대로62길 48 (우)05712
서울 노원구 동일로186길 53-50 1층 (우)01843
경기 남양주시 다산중앙로123번길 22-30 1층 (우)12248
경기 하남시 미사강변중앙로 180 힐스테이트 미사역 그랑파사쥬 1층 1026,1027호 (우)12913
경기 성남시 중원구 성남대로1148번길 13 2층 (우)13364
경기 군포시 군포로719번길 5 (우)15811
서울 송파구 오금로13길 12 1층 (우)05548
서울 서초구 동산로19길 35-1 (우)06781
경기 안양시 동안구 인덕원로24번길 44 1층 (우)13951
경기 광주시 초월읍 경충대로1127번길 21-5 (우)12736
경기 성남시 분당구 성남대로172번길 12 현대아리온 오피스텔 (우)13618
강원 원주시 토지길 31-8 1층 (우)26490
충남 천안시 서북구 1공단1길 52 센트하임프라자 1층 130~132호 (우)31104
충남 아산시 용화로47번길 2-10 (우)31574
충남 당진시 북문로1길 37-16 (우)31768
충남 천안시 서북구 성환읍 성환11길 13-6 (우)31016
경기 여주시 여양로233번길 11 (우)12639
충북 청주시 흥덕구 죽천로124번길 10 (우)28584
대전 서구 탄방로7번길 57 (우)35256
강원 속초시 먹거리3길 10 (우)24858
충남 논산시 시민로132번길 50-11 (우)32991
전북 완주군 삼례읍 녹색로 91 (우)55339
전북 완주군 이서면 오공로 11-19 1층 109

In [93]:
df

,name,average_rating,user,user_rating,comment,url,target
0,후계동,4.5,ㅇㅇ,5.0,❤️❤️❤️❤️❤️,https://place.map.kakao.com/1183957472,1
1,후계동,4.5,핑구,5.0,오래 해주세요🙏,https://place.map.kakao.com/1183957472,1
2,후계동,4.5,:ᴅ,3.0,웨이팅해서 먹을 맛은 아니에요… 닭보쌈은 보통이었고 비빔국수라고 해서 당연히 국물 ...,https://place.map.kakao.com/1183957472,0
3,후계동,4.5,Vin,5.0,와 여길 왜 이제알았죠! 너무맛있고 반찬 하나하나 다 맛있네요!! 자주 갈게요!,https://place.map.kakao.com/1183957472,1
4,후계동,4.5,손민영,5.0,오늘의 메뉴 너무 좋아요. 어쩜 갖가지 닭요리를 그렇게 잘하시죠?!! 이런 리뷰 잘...,https://place.map.kakao.com/1183957472,1
...,...,...,...,...,...,...,...
718,금복식당,4.0,가보나마나,4.0,기교를 부리지 않은 담백한 맛의 족발. 얇게 썰어 내 주시는데 부드럽고 적당히 탄력...,https://place.map.kakao.com/8207870,1
719,금복식당,4.0,빳빳한 모잘트,4.0,족발 맛있어요,https://place.map.kakao.com/8207870,1
720,금복식당,4.0,으쓱,4.0,,https://place.map.kakao.com/8207870,1
721,금복식당,4.0,ak,4.0,,https://place.map.kakao.com/8207870,1


In [98]:
# #target 값 설정
# y = df.shape[0]*[0]
# target = pd.Series(y, name='target')

# df = pd.concat([df, target], axis=1)
# df

df.info(verbose=True)
df["comment"].isnull().sum()
df["comment"][722]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 723 entries, 0 to 722
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            723 non-null    object 
 1   average_rating  723 non-null    float64
 2   user            723 non-null    object 
 3   user_rating     723 non-null    float64
 4   comment         723 non-null    object 
 5   url             723 non-null    object 
 6   target          723 non-null    object 
dtypes: float64(2), object(5)
memory usage: 39.7+ KB


''

In [99]:
df.to_csv("홍대_맛집_리뷰_데이터.csv")
df.to_excel("홍대_맛집_리뷰_데이터.xlsx")